# Imports y configuraciones básicas

Algunos imports que se utilizarán en el resto del notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from bwb.distributions import *
from bwb.utils import plot_list_of_draws, plot_histogram_from_points

import pandas as pd
import itertools

In [ ]:
import logging
log = logging.getLogger("bwb.distributions")
log.setLevel(logging.INFO)

Importamos los datos del archivo de Google Cloud

In [ ]:
data = np.load(r"..\data\face.npy")
data.shape

Haremos dos listas de caritas: una sin modificación, y otra con modificación.

In [ ]:
%%time

CEIL = 50
n_faces = len(data)
rng = np.random.default_rng(42)


# Builder para hacer las caritas
dist_draw_builder = DistributionDrawBuilder(
    seed=rng,
)

# Conjunto de caritas originales (distintos soportes)
faces_original = list()
for i in range(n_faces):
    arr = 255 - data[i, :].reshape(28, 28)
    dd_arr = dist_draw_builder.create_fromarray(arr)
    faces_original.append(dd_arr)
    
    
# Conjuntos de caritas con soporte no-nulo
dist_draw_builder.set_ceil(CEIL)
    
faces_modified = list()
for i in range(n_faces):
    arr = 255 - data[i, :].reshape(28, 28)
    dd_arr = dist_draw_builder.create_fromarray(arr)
    faces_modified.append(dd_arr)
    
len(faces_original), len(faces_modified)

`faces_original` son las caritas sin modificar el soporte, mientras que `faces_modified` es con los que tiene soporte no-nulo.

In [ ]:
plot_list_of_draws(faces_original)

In [ ]:
plot_list_of_draws(faces_modified)

Obtendremos la data $D = \{x_i\}_{i=1}^{n}\subseteq\mathcal{X}^n$, y probaremos los métodos para distintos valores de $n$.

In [ ]:
first_face = DistributionDraw.fromarray(
    255 - data[0, :].reshape(28, 28),
    seed=42
)

x = first_face.rvs(size=100)

plot_histogram_from_points(x, histplot_kwargs=dict(bins=28))
plt.show()

# Probando clase `MetropolisPosteriorPiN` datos originales

En esta sección se intentará simular la implementación del algoritmo de *Metropolis* programado en el notebook 05.

In [ ]:
%%time
mcmc = MetropolisPosteriorPiN(data=x[:10], models=faces_original, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

Y mostramos las caras en orden de la más común a la menos común

In [ ]:
plot_list_of_draws(mcmc.most_common())

In [ ]:
plot_histogram_from_points(x[:10], histplot_kwargs=dict(bins=28))
plt.show()

# Probando clase `AlternativeMetropolisPosteriorPiN` datos originales

En esta sección se intentará simular la implementación del algoritmo de *Metropolis* programado en el notebook 05.

In [ ]:
%%time
mcmc = AlternativeMetropolisPosteriorPiN(data=x[:10], models=faces_original, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

Y mostramos las caras en orden de la más común a la menos común

In [ ]:
plot_list_of_draws(mcmc.most_common())

# Probando clase `GibbsPosteriorPiN` datos originales

Hacemos lo mismo, pero para la clase `GibbsPosteriorPiN`.

In [ ]:
%%time
mcmc = GibbsPosteriorPiN(data=x[:10], models=faces_original, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
plot_list_of_draws(mcmc.most_common())

# Probando clase `AlternativeGibbsPosteriorPiN` datos originales

Hacemos lo mismo, pero para la clase `GibbsPosteriorPiN`.

In [ ]:
%%time
mcmc = AlternativeGibbsPosteriorPiN(data=x[:10], models=faces_original, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
plot_list_of_draws(mcmc.most_common())

# Probando clase `MetropolisPosteriorPiN` datos modificados

En esta sección se intentará simular el algoritmo de *Metropolis* programado en el notebook 06, el que posee los datos modificados con soporte no-nulo.

In [ ]:
%%time
mcmc = MetropolisPosteriorPiN(data=x[:10], models=faces_modified, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
plot_list_of_draws(mcmc.most_common())

In [ ]:
plot_histogram_from_points(x[:30], histplot_kwargs=dict(bins=28))
plt.show()

# Probando clase `GibbsPosteriorPiN` datos modificados

Probamos el algoritmo de Gibbs utilizando los datos modificados.

In [ ]:
%%time
mcmc = GibbsPosteriorPiN(data=x[:30], models=faces_modified, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

Notamos que el algoritmo tiene especial preferencia para la primera cara. Esto tiene sentido, pues es de donde se obtuvieron los datos. Sin embargo, por esta misma razón, se intentará correr el algoritmo con los otros datos, excluyendo la primera cara.

In [ ]:
plot_list_of_draws(mcmc.most_common())

# `GibbsPosteriorPiN` con los datos modificacos, sin el primer dibujo

In [ ]:
%%time
mcmc = GibbsPosteriorPiN(data=x[:30], models=faces_modified[1:], seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
plot_list_of_draws(mcmc.most_common())

Notamos que ahora los sampleos se encuentran más "equilibrados". Ahora nos interesaremos en un test de "estrés", querremos probar con $n$ alto.

# `GibbsPosteriorPiN` con $n=50$ sin la primera cara

In [ ]:
%%time
mcmc = GibbsPosteriorPiN(data=x[:50], models=faces_modified[1:], seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
faces_modified[0]

In [ ]:
plot_list_of_draws(mcmc.most_common())

# `GibbsPosteriorPiN` con $n=100$ sin la primera cara

In [ ]:
%%time
mcmc = GibbsPosteriorPiN(data=x, models=faces_modified[1:], seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
faces_modified[0]

In [ ]:
plot_list_of_draws(mcmc.most_common())

# `GibbsPosteriorPiN` con $n=100$ sin la primera cara

In [ ]:
%%time
mcmc = GibbsPosteriorPiN(data=x, models=faces_original, seed=rng)

Muestreamos a partir de de esta instancia.

In [ ]:
%%time
mcmc.rvs(size=1_000)
mcmc

Observamos el contador

In [ ]:
mcmc.counter.most_common()

In [ ]:
faces_modified[0]

In [ ]:
plot_list_of_draws(mcmc.most_common())

In [43]:
%%time
mcmc = GibbsPosteriorPiN(data=x, models=faces_modified[1:], seed=rng)

2023-01-24 15:14:35,551: INFO [bwb.distributions:243] init PosteriorPiN: n_data=100, n_models=161665
2023-01-24 15:14:35,553: INFO [bwb.distributions:460] Executing _first_step
2023-01-24 15:14:44,849: INFO [bwb.distributions:467] First model selected: 134615.


CPU times: total: 9.28 s
Wall time: 9.3 s


Muestreamos a partir de de esta instancia.

In [44]:
%%time
mcmc.rvs(size=1_000)
mcmc

2023-01-24 15:14:44,903: INFO [bwb.distributions:508] i = 1: last_i = 134615 -> next_i = 71038.
2023-01-24 15:14:44,928: INFO [bwb.distributions:508] i = 2: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:44,950: INFO [bwb.distributions:508] i = 3: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:44,988: INFO [bwb.distributions:508] i = 5: last_i = 71038 -> next_i = 148766.
2023-01-24 15:14:45,009: INFO [bwb.distributions:508] i = 6: last_i = 148766 -> next_i = 150907.
2023-01-24 15:14:45,047: INFO [bwb.distributions:508] i = 8: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:45,067: INFO [bwb.distributions:508] i = 9: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:45,128: INFO [bwb.distributions:508] i = 12: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:45,148: INFO [bwb.distributions:508] i = 13: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:45,169: INFO [bwb.distributions:508] i = 14: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:45,192: INFO [bwb.d

2023-01-24 15:14:47,191: INFO [bwb.distributions:508] i = 95: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:47,220: INFO [bwb.distributions:508] i = 96: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:47,244: INFO [bwb.distributions:508] i = 97: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:47,269: INFO [bwb.distributions:508] i = 98: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:47,294: INFO [bwb.distributions:508] i = 99: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:47,316: INFO [bwb.distributions:508] i = 100: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:47,341: INFO [bwb.distributions:508] i = 101: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:47,386: INFO [bwb.distributions:508] i = 103: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:47,406: INFO [bwb.distributions:508] i = 104: last_i = 150907 -> next_i = 38313.
2023-01-24 15:14:47,438: INFO [bwb.distributions:508] i = 105: last_i = 38313 -> next_i = 150907.
2023-01-24 15:14:47,460: 

2023-01-24 15:14:49,142: INFO [bwb.distributions:508] i = 185: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:49,163: INFO [bwb.distributions:508] i = 186: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:49,183: INFO [bwb.distributions:508] i = 187: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:49,222: INFO [bwb.distributions:508] i = 189: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:49,242: INFO [bwb.distributions:508] i = 190: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:49,262: INFO [bwb.distributions:508] i = 191: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:49,282: INFO [bwb.distributions:508] i = 192: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:49,321: INFO [bwb.distributions:508] i = 194: last_i = 150907 -> next_i = 148766.
2023-01-24 15:14:49,342: INFO [bwb.distributions:508] i = 195: last_i = 148766 -> next_i = 71038.
2023-01-24 15:14:49,365: INFO [bwb.distributions:508] i = 196: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:49

2023-01-24 15:14:51,424: INFO [bwb.distributions:508] i = 279: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:51,466: INFO [bwb.distributions:508] i = 281: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:51,507: INFO [bwb.distributions:508] i = 283: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:51,528: INFO [bwb.distributions:508] i = 284: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:51,548: INFO [bwb.distributions:508] i = 285: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:51,569: INFO [bwb.distributions:508] i = 286: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:51,590: INFO [bwb.distributions:508] i = 287: last_i = 71038 -> next_i = 65460.
2023-01-24 15:14:51,610: INFO [bwb.distributions:508] i = 288: last_i = 65460 -> next_i = 150907.
2023-01-24 15:14:51,632: INFO [bwb.distributions:508] i = 289: last_i = 150907 -> next_i = 127086.
2023-01-24 15:14:51,654: INFO [bwb.distributions:508] i = 290: last_i = 127086 -> next_i = 71038.
2023-01-24 15:14:51,

2023-01-24 15:14:53,567: INFO [bwb.distributions:508] i = 368: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:53,588: INFO [bwb.distributions:508] i = 369: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:53,611: INFO [bwb.distributions:508] i = 370: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:53,632: INFO [bwb.distributions:508] i = 371: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:53,654: INFO [bwb.distributions:508] i = 372: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:53,677: INFO [bwb.distributions:508] i = 373: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:53,698: INFO [bwb.distributions:508] i = 374: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:53,730: INFO [bwb.distributions:508] i = 375: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:53,751: INFO [bwb.distributions:508] i = 376: last_i = 150907 -> next_i = 71038.
2023-01-24 15:14:53,772: INFO [bwb.distributions:508] i = 377: last_i = 71038 -> next_i = 150907.
2023-01-24 15:14:53,

...

2023-01-24 15:15:01,739: INFO [bwb.distributions:508] i = 743: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:01,774: INFO [bwb.distributions:508] i = 745: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:01,793: INFO [bwb.distributions:508] i = 746: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:01,813: INFO [bwb.distributions:508] i = 747: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:01,834: INFO [bwb.distributions:508] i = 748: last_i = 150907 -> next_i = 148766.
2023-01-24 15:15:01,856: INFO [bwb.distributions:508] i = 749: last_i = 148766 -> next_i = 150907.
2023-01-24 15:15:01,875: INFO [bwb.distributions:508] i = 750: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:01,947: INFO [bwb.distributions:508] i = 754: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:01,965: INFO [bwb.distributions:508] i = 755: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:01,984: INFO [bwb.distributions:508] i = 756: last_i = 71038 -> next_i = 65460.
2023-01-24 15:15:02

2023-01-24 15:15:03,467: INFO [bwb.distributions:508] i = 833: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:03,487: INFO [bwb.distributions:508] i = 834: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:03,506: INFO [bwb.distributions:508] i = 835: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:03,525: INFO [bwb.distributions:508] i = 836: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:03,544: INFO [bwb.distributions:508] i = 837: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:03,563: INFO [bwb.distributions:508] i = 838: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:03,581: INFO [bwb.distributions:508] i = 839: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:03,601: INFO [bwb.distributions:508] i = 840: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:03,619: INFO [bwb.distributions:508] i = 841: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:03,638: INFO [bwb.distributions:508] i = 842: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:03,

2023-01-24 15:15:05,223: INFO [bwb.distributions:508] i = 923: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:05,243: INFO [bwb.distributions:508] i = 924: last_i = 71038 -> next_i = 148766.
2023-01-24 15:15:05,264: INFO [bwb.distributions:508] i = 925: last_i = 148766 -> next_i = 71038.
2023-01-24 15:15:05,286: INFO [bwb.distributions:508] i = 926: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:05,306: INFO [bwb.distributions:508] i = 927: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:05,325: INFO [bwb.distributions:508] i = 928: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:05,345: INFO [bwb.distributions:508] i = 929: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:05,364: INFO [bwb.distributions:508] i = 930: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:05,384: INFO [bwb.distributions:508] i = 931: last_i = 150907 -> next_i = 71038.
2023-01-24 15:15:05,407: INFO [bwb.distributions:508] i = 932: last_i = 71038 -> next_i = 150907.
2023-01-24 15:15:05,

CPU times: total: 22.1 s
Wall time: 21.8 s


GibbsPosteriorPiN(n_data=100, n_models=161665, n_samples=1000, last_i=71038, likelihood_sum=9.0517e-267)

Observamos el contador

In [45]:
mcmc.counter.most_common()

[(150907, 428),
 (71038, 424),
 (136493, 68),
 (148766, 33),
 (65460, 28),
 (127086, 5),
 (38313, 3),
 (83639, 3),
 (134615, 2),
 (128011, 2),
 (91946, 2),
 (114247, 1),
 (91382, 1),
 (97267, 1)]

In [46]:
faces_modified[0]

In [47]:
plot_list_of_draws(mcmc.most_common())

# `GibbsPosteriorPiN` con $n=100$ sin la primera cara

In [48]:
%%time
mcmc = GibbsPosteriorPiN(data=x, models=faces_original, seed=rng)

2023-01-24 15:15:06,850: INFO [bwb.distributions:243] init PosteriorPiN: n_data=100, n_models=161666
2023-01-24 15:15:06,852: INFO [bwb.distributions:460] Executing _first_step
C:\Users\asoln\PycharmProjects\PersonalProjects\Bayesian-Learning-with-Wasserstein-Barycenters\bwb\distributions.py:263: UserWarning: The likelihood support has only one element.
  warnings.warn("The likelihood support has only one element.")
2023-01-24 15:15:07,621: INFO [bwb.distributions:467] First model selected: 0.


CPU times: total: 672 ms
Wall time: 772 ms


Muestreamos a partir de de esta instancia.

In [49]:
%%time
mcmc.rvs(size=1_000)
mcmc

CPU times: total: 19.3 s
Wall time: 19.5 s


GibbsPosteriorPiN(n_data=100, n_models=161666, n_samples=1000, last_i=0, likelihood_sum=3.9950e-238)

Observamos el contador

In [50]:
mcmc.counter.most_common()

[(0, 1001)]

In [51]:
faces_modified[0]

In [52]:
plot_list_of_draws(mcmc.most_common())